In [1]:
import glob
fname = glob.glob("./drive/My Drive/機不可失/nier/*.csv")
fname

['./drive/My Drive/機不可失/nier/ptt_iphone.csv',
 './drive/My Drive/機不可失/nier/ptt_pixel.csv',
 './drive/My Drive/機不可失/nier/ptt_iphone_c.csv',
 './drive/My Drive/機不可失/nier/ptt_google.csv',
 './drive/My Drive/機不可失/nier/ptt_samsung.csv',
 './drive/My Drive/機不可失/nier/ptt_samsung_c.csv',
 './drive/My Drive/機不可失/nier/ptt_mi_c.csv',
 './drive/My Drive/機不可失/nier/ptt_sony.csv',
 './drive/My Drive/機不可失/nier/ptt_sony_c.csv']

In [2]:
import pandas as pd
fname = pd.read_csv("./drive/My Drive/機不可失/nier/ptt_iphone.csv")
fname2 = pd.read_csv("./drive/My Drive/機不可失/nier/ptt_iphone_c.csv")
print("第一個資料", fname.shape)
print("第二個資料", fname2.shape)

第一個資料 (647, 6)
第二個資料 (2209, 6)


In [3]:
df = pd.concat([fname, fname2], axis=0)
df

,ID,看板,標題,時間,分數,內容
0,z1976 (z1976),MobileComm,[問題] Apple watch LTE可以當行動基地台嗎?,Tue May 12 14:18:29 2020,6,\n我本身是軟體工程師\n\n因公司的網路政策讓我們在取得很多開發資源時窒礙難行\n\n我都...
1,kyle5241 (Kyle Korver),MobileComm,[情報] 12X 的新Apple TV即將發表,Fri May 8 00:06:29 2020,15,\nhttps://mashable.com/article/new-apple-tv-le...
2,olmtw (支持htc，支持台灣貨),MobileComm,[新聞] 實用性不輸 Apple Watch！外媒評 5 款最,Mon May 4 14:44:40 2020,20,\n實用性不輸 Apple Watch！外媒評 5 款最佳平價智慧手環\n\n文／記者黃敬淳...
3,peterlin495 (專業果粉O'_'O),MobileComm,[新聞] LINE換機轉移有新招 用Apple帳號登入不再,Sun May 3 21:31:31 2020,15,\nLINE換機轉移有新招 用Apple帳號登入不再怕失敗\n2020-05-03 19:0...
4,mcharuko (象),MobileComm,Re: [問題] asus 5z / apple se2 選擇,Thu Apr 30 23:11:07 2020,8,\n※ 引述《p81827512 (inverse)》之銘言：\n\n1. zf3 如果還堪...
...,...,...,...,...,...,...
2204,winall (winall),MobileComm,Re: [新聞]蘋果賺很大 超過對手總合,Thu Oct 31 13:17:08 2013,35,"\n\n為何大家不想一個問題\n\n三星賣 1.2 億隻 , Apple賣 3000萬隻\n..."
2205,ww5555 (東興鳥鴉-躍楊),MobileComm,Re: [新聞] 智慧機滿意度、蘋果十連霸,Sat Oct 19 09:17:36 2013,33,\n※ 引述《Lsamia (samia)》之銘言：\n: http://0rz.tw/o6...
2206,Rigaudon (警察都是垃圾),MobileComm,Re: [心得] 真的以為用蘋果就很潮嗎...,Tue Nov 9 06:54:32 2010,4,\n真怪 人家使用者自己都沒有意見了 偏偏旁觀者意見一大堆\n\n不能看flash\n不能剪...
2207,nick1119 (Uncle C),MobileComm,[情報] 蘋果有意自行開發手機晶片,Fri May 1 10:40:08 2009,1,\n資訊整理：http://www.digi-cit.com/cit/viewthread....


In [4]:
# 分詞, 數次數, 把慣用係數乘上去, 總分排順序, jieba參考：https://github.com/fxsjy/jieba
# 標題處理, 
import jieba, jieba.analyse
# 載入要得辭典 和不要得辭典
jieba.load_userdict("./drive/My Drive/機不可失/nier/mydict.txt")
jieba.analyse.set_stop_words("./drive/My Drive/機不可失/nier/stopwords.txt") 
def jana(s):
  s = s.lower()
  return jieba.analyse.extract_tags(s, 100, withWeight=True)
#keyword = fname["標題"].apply(jana)
#keyword

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.690 seconds.
Prefix dict has been built successfully.


In [5]:
# 內文處理
keyartical = df["內容"].apply(jana)
keyartical[0]

0    [(05, 1.0131158900762711), (12, 0.851017347664...
0    [(05, 0.3136819461402332), (13, 0.209121297426...
Name: 內容, dtype: object

In [6]:
# 找出內文裡面有 list k 這些字的文章, 有的為 True
def check(s):
  k = ["電量", "充電速度", "使用時間", "QC3.0", "無線充電", "快充", "閃充", "電池"]
  if any(i in s for i in k):
    return s.index 
ide = df["內容"].apply(check)
# 把內容裡面有 list k 裡面的文字的留下, 總共有 154筆資料
ide[ide.isna() != 1]

0       <built-in method index of str object at 0x3c40...
4       <built-in method index of str object at 0x3dd6...
7       <built-in method index of str object at 0x3d44...
9       <built-in method index of str object at 0x3d3b...
10      <built-in method index of str object at 0x3c07...
                              ...                        
2190    <built-in method index of str object at 0x7b24...
2196    <built-in method index of str object at 0x7cae...
2197    <built-in method index of str object at 0x7ca1...
2199    <built-in method index of str object at 0x7c8a...
2206    <built-in method index of str object at 0x7d0c...
Name: 內容, Length: 853, dtype: object

In [7]:
# 將有關鍵字的這些文章給一個標籤: 續航力 # 把 true 變成 0, false 變成 1
import numpy as np
k = np.array(ide.isna())
k = k.astype(int)
sum = 0
for i in range(len(k)):
  if k[i] == 0:
    k[i] = 1
    sum +=1
  else:
    k[i] =0
df["續航力"] = k
df

,ID,看板,標題,時間,分數,內容,續航力
0,z1976 (z1976),MobileComm,[問題] Apple watch LTE可以當行動基地台嗎?,Tue May 12 14:18:29 2020,6,\n我本身是軟體工程師\n\n因公司的網路政策讓我們在取得很多開發資源時窒礙難行\n\n我都...,1
1,kyle5241 (Kyle Korver),MobileComm,[情報] 12X 的新Apple TV即將發表,Fri May 8 00:06:29 2020,15,\nhttps://mashable.com/article/new-apple-tv-le...,0
2,olmtw (支持htc，支持台灣貨),MobileComm,[新聞] 實用性不輸 Apple Watch！外媒評 5 款最,Mon May 4 14:44:40 2020,20,\n實用性不輸 Apple Watch！外媒評 5 款最佳平價智慧手環\n\n文／記者黃敬淳...,0
3,peterlin495 (專業果粉O'_'O),MobileComm,[新聞] LINE換機轉移有新招 用Apple帳號登入不再,Sun May 3 21:31:31 2020,15,\nLINE換機轉移有新招 用Apple帳號登入不再怕失敗\n2020-05-03 19:0...,0
4,mcharuko (象),MobileComm,Re: [問題] asus 5z / apple se2 選擇,Thu Apr 30 23:11:07 2020,8,\n※ 引述《p81827512 (inverse)》之銘言：\n\n1. zf3 如果還堪...,1
...,...,...,...,...,...,...,...
2204,winall (winall),MobileComm,Re: [新聞]蘋果賺很大 超過對手總合,Thu Oct 31 13:17:08 2013,35,"\n\n為何大家不想一個問題\n\n三星賣 1.2 億隻 , Apple賣 3000萬隻\n...",0
2205,ww5555 (東興鳥鴉-躍楊),MobileComm,Re: [新聞] 智慧機滿意度、蘋果十連霸,Sat Oct 19 09:17:36 2013,33,\n※ 引述《Lsamia (samia)》之銘言：\n: http://0rz.tw/o6...,0
2206,Rigaudon (警察都是垃圾),MobileComm,Re: [心得] 真的以為用蘋果就很潮嗎...,Tue Nov 9 06:54:32 2010,4,\n真怪 人家使用者自己都沒有意見了 偏偏旁觀者意見一大堆\n\n不能看flash\n不能剪...,1
2207,nick1119 (Uncle C),MobileComm,[情報] 蘋果有意自行開發手機晶片,Fri May 1 10:40:08 2009,1,\n資訊整理：http://www.digi-cit.com/cit/viewthread....,0


In [8]:
# 標籤：相機
def check(s):
  k2 = ["畫素", "廣角", "錄影", "光圈", "多鏡頭", "對焦", "夜拍", "自拍", "遠景", "近景", "修圖", "變焦"]
  if any(i in s for i in k2):
    return s.index 
ide = df["內容"].apply(check)
# 把內容裡面有 list k 裡面的文字的留下
#ide[ide.isna() != 1]
k = np.array(ide.isna())
k = k.astype(int)
sum = 0
for i in range(len(k)):
  if k[i] == 0:
    k[i] = 1
    sum +=1
  else:
    k[i] =0
df["相機"] = k
df

,ID,看板,標題,時間,分數,內容,續航力,相機
0,z1976 (z1976),MobileComm,[問題] Apple watch LTE可以當行動基地台嗎?,Tue May 12 14:18:29 2020,6,\n我本身是軟體工程師\n\n因公司的網路政策讓我們在取得很多開發資源時窒礙難行\n\n我都...,1,0
1,kyle5241 (Kyle Korver),MobileComm,[情報] 12X 的新Apple TV即將發表,Fri May 8 00:06:29 2020,15,\nhttps://mashable.com/article/new-apple-tv-le...,0,0
2,olmtw (支持htc，支持台灣貨),MobileComm,[新聞] 實用性不輸 Apple Watch！外媒評 5 款最,Mon May 4 14:44:40 2020,20,\n實用性不輸 Apple Watch！外媒評 5 款最佳平價智慧手環\n\n文／記者黃敬淳...,0,0
3,peterlin495 (專業果粉O'_'O),MobileComm,[新聞] LINE換機轉移有新招 用Apple帳號登入不再,Sun May 3 21:31:31 2020,15,\nLINE換機轉移有新招 用Apple帳號登入不再怕失敗\n2020-05-03 19:0...,0,0
4,mcharuko (象),MobileComm,Re: [問題] asus 5z / apple se2 選擇,Thu Apr 30 23:11:07 2020,8,\n※ 引述《p81827512 (inverse)》之銘言：\n\n1. zf3 如果還堪...,1,1
...,...,...,...,...,...,...,...,...
2204,winall (winall),MobileComm,Re: [新聞]蘋果賺很大 超過對手總合,Thu Oct 31 13:17:08 2013,35,"\n\n為何大家不想一個問題\n\n三星賣 1.2 億隻 , Apple賣 3000萬隻\n...",0,0
2205,ww5555 (東興鳥鴉-躍楊),MobileComm,Re: [新聞] 智慧機滿意度、蘋果十連霸,Sat Oct 19 09:17:36 2013,33,\n※ 引述《Lsamia (samia)》之銘言：\n: http://0rz.tw/o6...,0,0
2206,Rigaudon (警察都是垃圾),MobileComm,Re: [心得] 真的以為用蘋果就很潮嗎...,Tue Nov 9 06:54:32 2010,4,\n真怪 人家使用者自己都沒有意見了 偏偏旁觀者意見一大堆\n\n不能看flash\n不能剪...,1,0
2207,nick1119 (Uncle C),MobileComm,[情報] 蘋果有意自行開發手機晶片,Fri May 1 10:40:08 2009,1,\n資訊整理：http://www.digi-cit.com/cit/viewthread....,0,0


In [9]:
# 標籤：效能
def check(s):
  k3 = ["處理器", "cpu", "高通", "A13", "A12", "遊戲", "多核心", "snapdragon", "klein", "高效能", "流暢"]
  if any(i in s for i in k3):
    return s.index 
ide = df["內容"].apply(check)
k = np.array(ide.isna())
k = k.astype(int)
sum = 0
for i in range(len(k)):
  if k[i] == 0:
    k[i] = 1
    sum +=1
  else:
    k[i] =0
df["效能"] = k
df

,ID,看板,標題,時間,分數,內容,續航力,相機,效能
0,z1976 (z1976),MobileComm,[問題] Apple watch LTE可以當行動基地台嗎?,Tue May 12 14:18:29 2020,6,\n我本身是軟體工程師\n\n因公司的網路政策讓我們在取得很多開發資源時窒礙難行\n\n我都...,1,0,0
1,kyle5241 (Kyle Korver),MobileComm,[情報] 12X 的新Apple TV即將發表,Fri May 8 00:06:29 2020,15,\nhttps://mashable.com/article/new-apple-tv-le...,0,0,1
2,olmtw (支持htc，支持台灣貨),MobileComm,[新聞] 實用性不輸 Apple Watch！外媒評 5 款最,Mon May 4 14:44:40 2020,20,\n實用性不輸 Apple Watch！外媒評 5 款最佳平價智慧手環\n\n文／記者黃敬淳...,0,0,0
3,peterlin495 (專業果粉O'_'O),MobileComm,[新聞] LINE換機轉移有新招 用Apple帳號登入不再,Sun May 3 21:31:31 2020,15,\nLINE換機轉移有新招 用Apple帳號登入不再怕失敗\n2020-05-03 19:0...,0,0,1
4,mcharuko (象),MobileComm,Re: [問題] asus 5z / apple se2 選擇,Thu Apr 30 23:11:07 2020,8,\n※ 引述《p81827512 (inverse)》之銘言：\n\n1. zf3 如果還堪...,1,1,1
...,...,...,...,...,...,...,...,...,...
2204,winall (winall),MobileComm,Re: [新聞]蘋果賺很大 超過對手總合,Thu Oct 31 13:17:08 2013,35,"\n\n為何大家不想一個問題\n\n三星賣 1.2 億隻 , Apple賣 3000萬隻\n...",0,0,0
2205,ww5555 (東興鳥鴉-躍楊),MobileComm,Re: [新聞] 智慧機滿意度、蘋果十連霸,Sat Oct 19 09:17:36 2013,33,\n※ 引述《Lsamia (samia)》之銘言：\n: http://0rz.tw/o6...,0,0,0
2206,Rigaudon (警察都是垃圾),MobileComm,Re: [心得] 真的以為用蘋果就很潮嗎...,Tue Nov 9 06:54:32 2010,4,\n真怪 人家使用者自己都沒有意見了 偏偏旁觀者意見一大堆\n\n不能看flash\n不能剪...,1,0,1
2207,nick1119 (Uncle C),MobileComm,[情報] 蘋果有意自行開發手機晶片,Fri May 1 10:40:08 2009,1,\n資訊整理：http://www.digi-cit.com/cit/viewthread....,0,0,1


In [10]:
# 標籤：遊戲
def check(s):
  k4 = ["絕對求生", "吃雞", "高偵數", "lag", "崩壞3rd", "傳說對決", "遊戲效能", "遊戲體驗", "rog phone", "電量", "ram"]
  if any(i in s for i in k4):
    return s.index 
ide = df["內容"].apply(check)
k = np.array(ide.isna())
k = k.astype(int)
sum = 0
for i in range(len(k)):
  if k[i] == 0:
    k[i] = 1
    sum +=1
  else:
    k[i] =0
df["遊戲"] = k
df

,ID,看板,標題,時間,分數,內容,續航力,相機,效能,遊戲
0,z1976 (z1976),MobileComm,[問題] Apple watch LTE可以當行動基地台嗎?,Tue May 12 14:18:29 2020,6,\n我本身是軟體工程師\n\n因公司的網路政策讓我們在取得很多開發資源時窒礙難行\n\n我都...,1,0,0,1
1,kyle5241 (Kyle Korver),MobileComm,[情報] 12X 的新Apple TV即將發表,Fri May 8 00:06:29 2020,15,\nhttps://mashable.com/article/new-apple-tv-le...,0,0,1,0
2,olmtw (支持htc，支持台灣貨),MobileComm,[新聞] 實用性不輸 Apple Watch！外媒評 5 款最,Mon May 4 14:44:40 2020,20,\n實用性不輸 Apple Watch！外媒評 5 款最佳平價智慧手環\n\n文／記者黃敬淳...,0,0,0,0
3,peterlin495 (專業果粉O'_'O),MobileComm,[新聞] LINE換機轉移有新招 用Apple帳號登入不再,Sun May 3 21:31:31 2020,15,\nLINE換機轉移有新招 用Apple帳號登入不再怕失敗\n2020-05-03 19:0...,0,0,1,1
4,mcharuko (象),MobileComm,Re: [問題] asus 5z / apple se2 選擇,Thu Apr 30 23:11:07 2020,8,\n※ 引述《p81827512 (inverse)》之銘言：\n\n1. zf3 如果還堪...,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
2204,winall (winall),MobileComm,Re: [新聞]蘋果賺很大 超過對手總合,Thu Oct 31 13:17:08 2013,35,"\n\n為何大家不想一個問題\n\n三星賣 1.2 億隻 , Apple賣 3000萬隻\n...",0,0,0,0
2205,ww5555 (東興鳥鴉-躍楊),MobileComm,Re: [新聞] 智慧機滿意度、蘋果十連霸,Sat Oct 19 09:17:36 2013,33,\n※ 引述《Lsamia (samia)》之銘言：\n: http://0rz.tw/o6...,0,0,0,0
2206,Rigaudon (警察都是垃圾),MobileComm,Re: [心得] 真的以為用蘋果就很潮嗎...,Tue Nov 9 06:54:32 2010,4,\n真怪 人家使用者自己都沒有意見了 偏偏旁觀者意見一大堆\n\n不能看flash\n不能剪...,1,0,1,0
2207,nick1119 (Uncle C),MobileComm,[情報] 蘋果有意自行開發手機晶片,Fri May 1 10:40:08 2009,1,\n資訊整理：http://www.digi-cit.com/cit/viewthread....,0,0,1,0


In [11]:
# 標籤：螢幕
def check(s):
  k5 = ["挖屏", "曲面", "大螢幕", "小螢幕", "qhd", "oled", "rgb", "尺寸", "smiled", "螢幕大小", "ips"]
  if any(i in s for i in k5):
    return s.index 
ide = df["內容"].apply(check)
k = np.array(ide.isna())
k = k.astype(int)
sum = 0
for i in range(len(k)):
  if k[i] == 0:
    k[i] = 1
    sum +=1
  else:
    k[i] =0
df["螢幕"] = k
df

,ID,看板,標題,時間,分數,內容,續航力,相機,效能,遊戲,螢幕
0,z1976 (z1976),MobileComm,[問題] Apple watch LTE可以當行動基地台嗎?,Tue May 12 14:18:29 2020,6,\n我本身是軟體工程師\n\n因公司的網路政策讓我們在取得很多開發資源時窒礙難行\n\n我都...,1,0,0,1,0
1,kyle5241 (Kyle Korver),MobileComm,[情報] 12X 的新Apple TV即將發表,Fri May 8 00:06:29 2020,15,\nhttps://mashable.com/article/new-apple-tv-le...,0,0,1,0,0
2,olmtw (支持htc，支持台灣貨),MobileComm,[新聞] 實用性不輸 Apple Watch！外媒評 5 款最,Mon May 4 14:44:40 2020,20,\n實用性不輸 Apple Watch！外媒評 5 款最佳平價智慧手環\n\n文／記者黃敬淳...,0,0,0,0,0
3,peterlin495 (專業果粉O'_'O),MobileComm,[新聞] LINE換機轉移有新招 用Apple帳號登入不再,Sun May 3 21:31:31 2020,15,\nLINE換機轉移有新招 用Apple帳號登入不再怕失敗\n2020-05-03 19:0...,0,0,1,1,0
4,mcharuko (象),MobileComm,Re: [問題] asus 5z / apple se2 選擇,Thu Apr 30 23:11:07 2020,8,\n※ 引述《p81827512 (inverse)》之銘言：\n\n1. zf3 如果還堪...,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
2204,winall (winall),MobileComm,Re: [新聞]蘋果賺很大 超過對手總合,Thu Oct 31 13:17:08 2013,35,"\n\n為何大家不想一個問題\n\n三星賣 1.2 億隻 , Apple賣 3000萬隻\n...",0,0,0,0,1
2205,ww5555 (東興鳥鴉-躍楊),MobileComm,Re: [新聞] 智慧機滿意度、蘋果十連霸,Sat Oct 19 09:17:36 2013,33,\n※ 引述《Lsamia (samia)》之銘言：\n: http://0rz.tw/o6...,0,0,0,0,0
2206,Rigaudon (警察都是垃圾),MobileComm,Re: [心得] 真的以為用蘋果就很潮嗎...,Tue Nov 9 06:54:32 2010,4,\n真怪 人家使用者自己都沒有意見了 偏偏旁觀者意見一大堆\n\n不能看flash\n不能剪...,1,0,1,0,0
2207,nick1119 (Uncle C),MobileComm,[情報] 蘋果有意自行開發手機晶片,Fri May 1 10:40:08 2009,1,\n資訊整理：http://www.digi-cit.com/cit/viewthread....,0,0,1,0,0


In [0]:
# 存檔
result = fname.drop(["ID", "看板", "時間", "分數"], axis=1)
result.to_csv("ptt_iphone_r_total.csv", encoding="utf-8", index=False)